# Разбор названий товаров

В этом примере показано, как с помощью Yargy-парсера привести к нормальному виду названия товаров, например:
```
Продам xbox 360 500 гб
{
  "console": {
    "name": "Xbox",
    "model": "360"
  },
  "attributes": [
    {
      "volume": 500,
      "unit": "GB"
    }
  ]
}

Пульт PlayStation3
{
  "accessory": {
    "type": "пульт"
  },
  "console": {
    "name": "PlayStation",
    "model": "3"
  }
}
```

Для чего может быть нужна такая технология:
1. Семантический поиск. Пользователь пишет в запросе "плойка", находятся товары "PS4 PRO", "Sony PlayStation 4 500Gb Slim" хотя общих подстрок у них нет
2. Метчинг товаров. Маркетплейсы типа market.ya.ru, goods.ru собирают информацию о товарах из разных интернет-магазинов. Далеко не всегда магазины знают штрих код товара, приходится как-то понимать, что "iPhone 4 white" это то же самое что "айфон 4 белый"

В примере рассматривается только одна категория "игровые приставки" с сайта Авито. Это значительно упрощает задачу. С помощью Yargy-парсера можно написать программу, которая будет понимать и другие категории товаров, но эта задача выходит за рамки данного примера.

![](thumb.png)

# Data

In [1]:
from random import seed, sample


def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')
            
            
lines = list(load_lines('titles.txt'))


seed(10)
for line in sample(lines, 20):
    print(line)

Пульт PlayStation3
Сонька 3
Xbox one
Sony PlayStation 4 500Gb Slim
Новый жесткий кейс psp
Ps3 slim прошитая rebug 4.81 + fifa 18
Беспроводной геймпад для Xbox One
Sony PlayStation 3 Super Slim 500gb
PS 4 white 500g
PS3 Sony PlayStation 3 Super Slim 500 GB
Sony PlayStation 3 Super Slim
PSP
Продам xbox 360 500 гб
Sony Playstation 3 slim (ps 3 slim) на запчасти
SonyPlaystation 4 500gb(белая)
PS4 PRO
PSP E1008, 32 Гб, прошитая, 73 игры. Вариант 2
Sony Playstation 3 Slim, 250 GB, Прошитая
PSP - Игровая карманная приставка
Sega


# Grammar

In [2]:
from yargy import (
    Parser,
    rule, or_, and_, not_,
)
from yargy.predicates import (
    caseless, type, gram, normalized,
    in_, in_caseless, dictionary
)
from yargy.pipelines import (
    caseless_pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp


def test(rule, *tests):
    parser = Parser(rule)
    for line, etalon in tests:
        match = parser.match(line)
        assert match, line
        guess = match.fact
        assert etalon == guess, guess

## Model, vendor

In [3]:
Console = fact(
    'Console',
    ['vendor', 'name', 'model', 'version']
)

### PS

In [4]:
NAME = morph_pipeline([
    'playstation',
    'play station',
    'ps',
    'плейстейшен',
    'сонька',
    'плестейшен',
    'плeйстeшн',
    'пс',
    'плойка'
]).interpretation(
    Console.name.const('PlayStation')
)

MODEL = or_(
    in_('1234'),
    in_caseless({
        'one',
        'vr'
    })
).interpretation(
    Console.model.custom(str.upper)
)

VENDOR = in_caseless([
    'sony',
    'сони',
    'soni'
]).interpretation(
    Console.vendor.const('Sony')
)

VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

VERSION = caseless_pipeline(VERSIONS).interpretation(
    Console.version.normalized().custom(VERSIONS.get)
)

SEP = in_({'-', '/'})

PLAYSTATION = rule(
    VENDOR.optional(),
    NAME,
    SEP.optional(),
    MODEL.optional(),
    VERSION.optional()
).interpretation(
    Console
)

test(
    PLAYSTATION,
    ['Sony PlayStation 3 Super Slim', Console(vendor='Sony', name='PlayStation', model='3', version='SuperSlim')],
    ['Ps vr slim', Console(name='PlayStation', model='VR', version='Slim')]
)

In [5]:
NAME = caseless('psp').interpretation(
    Console.name.const('PSP')
)

MODEL = in_caseless({
    'vita',
    'slim'
}).interpretation(
    Console.model.custom(str.capitalize)
)

PSP = rule(
    VENDOR.optional(),
    NAME,
    SEP.optional(),
    MODEL.optional()
).interpretation(
    Console
)

test(
    PSP,
    ['PSP', Console(name='PSP')],
    ['Sony Psp vita', Console(vendor='Sony', name='PSP', model='Vita')]
)

### Xbox

In [6]:
NAME = caseless_pipeline([
    'x box',
    'x-box',
    'xbox',
    'хbox',  # х русская
    'х-бох',
    'х-вох',
    'хбокс',
    'хбох',
    'хвох',
    'икс бокс',
    'иксбокс',
]).interpretation(
    Console.name.const('Xbox')
)

MODEL = in_caseless({
    '360',
    'one',
    'original'
}).interpretation(
    Console.model.custom(str.capitalize)
)

VERSION = in_caseless({
    'slim',
    's',
    'e',
    'fat'
}).interpretation(
    Console.version.custom(str.capitalize)
)

XBOX = rule(
    NAME,
    SEP.optional(),
    MODEL.optional(),
    VERSION.optional()
).interpretation(
    Console
)

test(
    XBOX,
    ['xbox 360', Console(name='Xbox', model='360')]
)

### Dendy

In [7]:
NAME = in_caseless({
    'dendy',
    'денди'
}).interpretation(
    Console.name.const('Dendy')
)

MODELS = {
    'giga drive': 'Giga Drive',
    'gigadrive': 'Giga Drive',
    'firecore': 'Firecore',
    'garfield': 'Garfield',
    'junior': 'Junior',
    'tomb raider': 'Tomb Raider',
    'subor': 'Subor',
    'pocket': 'Pocket'
}

MODEL = caseless_pipeline(MODELS).interpretation(
    Console.model.normalized().custom(MODELS.get)
)

DENDY = rule(
    NAME,
    SEP.optional(),
    MODEL.optional()
).interpretation(
    Console
)

test(
    DENDY,
    ['Dendy Tomb Raider', Console(name='Dendy', model='Tomb Raider')],
    ['Dendy Pocket', Console(name='Dendy', model='Pocket')]
)

### Nintendo

In [8]:
VENDOR = in_caseless({
    'nintendo',
    'нинтендо'
}).interpretation(
    Console.vendor.const('Nintendo')
)

NAMES = {
    'ds': 'DS',
    'switch': 'Switch',
    'wii': 'Wii',
    'свич': 'Switch',
    '3ds': '3DS'
}

NAME = caseless_pipeline(NAMES).interpretation(
    Console.name.normalized().custom(NAMES.get)
)

VERSION = in_caseless({
    'lite',
    'mini',
    'XL'
}).interpretation(
    Console.version.custom(str.upper)
)

NINTENDO = rule(
    VENDOR,
    SEP.optional(),
    NAME.optional(),
    VERSION.optional()
).interpretation(
    Console
)


test(
    NINTENDO,
    ['Nintendo Wii', Console(vendor='Nintendo', name='Wii')],
    ['Нинтендо Свич', Console(vendor='Nintendo', name='Switch')],
    ['Nintendo 3ds XL', Console(vendor='Nintendo', name='3DS', version='XL')],
)

### Sega

In [9]:
VENDOR = in_caseless({
    'sega',
    'сега'
}).interpretation(
    Console.vendor.const('Sega')
)

NAMES = {
    'drive': 'Drive',
    'megadrive': 'MegaDrive',
    'mega drive': 'MegaDrive',
    'super drive': 'SuperDrive',
    'superdrive': 'SuperDrive',
    'dreamcast': 'Dreamcast'
}

NAME = caseless_pipeline(NAMES).interpretation(
    Console.name.normalized().custom(NAMES.get)
)

SEGA = rule(
    VENDOR,
    NAME.optional()
).interpretation(
    Console
)

test(
    SEGA,
    ['Sega Super Drive', Console(vendor='Sega', name='SuperDrive')]
)

### Console

In [10]:
CONSOLE = or_(
    PLAYSTATION,
    PSP,
    XBOX,
    DENDY,
    NINTENDO,
    SEGA
)

PREFIX = morph_pipeline([
    'приставка',
    'консоль',
    'игровая консоль',
    'игровая приставка',
    'приставка игровая',
    'console',
])

CONSOLE = or_(
    PREFIX,
    rule(
        PREFIX.optional(),
        CONSOLE
    )
).interpretation(
    Console
)


test(
    CONSOLE,
    ['Игровая приставка Sega Dreamcast', Console(vendor='Sega', name='Dreamcast')],
    ['Игровая консоль Sony ps4', Console(vendor='Sony', name='PlayStation', model='4')]
)

## Games

In [11]:
Games = fact(
    'Games',
    ['number']
)


INT = type('INT').interpretation(
    interp.custom(int)
)

NUMRS = {
    'один': 1,
    'два': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9,
    'десять': 10,
}

NUMR = dictionary(NUMRS).interpretation(
    interp.normalized().custom(NUMRS.get)
)

MANY = dictionary({
    'много',
    'куча'
}).interpretation(
    interp.const(float('inf'))
)

NUMBER = or_(
    INT,
    NUMR,
    MANY
).interpretation(
    Games.number
)

WORDS = morph_pipeline([
    'игра',
    'набор игр',
    'диск с игрой',
    'диск',
    'game',
    'games'
])

GAMES = rule(
    NUMBER.optional(),
    WORDS
).interpretation(
    Games
)


test(
    GAMES,
    ['много игр', Games(number=float('inf'))],
    ['5 дисков', Games(number=5)],
    ['два диска с играми', Games(number=2)]
)

## Accessory

In [12]:
Accessory = fact(
    'Accessory',
    ['number', 'type']
)


TYPES = {
    'camera': 'камера',
    'камера': 'камера',

    'gamepad': 'джойстик',
    'gamepads': 'джойстик',
    'joystick': 'джойстик',
    'беспроводной геймпад': 'джойстик',
    'беспроводной пульт': 'джойстик',
    'геймпад джойстик': 'джойстик',
    'геймпад': 'джойстик',
    'джойстик': 'джойстик',
    'джостик': 'джойстик',
    'контроллер': 'джойстик',

    'kinect': 'kinect',
    'kinekt': 'kinect',
    'кинект': 'kinect',

    'move': 'move',
    'мув': 'move',

    'кабель': 'кабель',
    'композитный кабель': 'кабель',

    'чехол': 'чехол',
    'защитный чехол': 'чехол',
    'кейс': 'чехол',

    'dualshock': 'dualshock',
    'аксессуар': 'аксессуар',
    'блок питания': 'блок питания',
    'гарнитура': 'гарнитура',
    'педали': 'педали',
    'пульт': 'пульт',
    'руль': 'руль',
    'сменная панель': 'сменная панель',
    'карта памяти': 'карта памяти',
    'наушники': 'наушники',
}

TYPE = morph_pipeline(TYPES).interpretation(
    Accessory.type.normalized().custom(TYPES.get)
)

NUMBER = or_(
    INT,
    NUMR
).interpretation(
    Accessory.number
)

ACCESSORY = rule(
    NUMBER.optional(),
    TYPE
).interpretation(
    Accessory
)

test(
    ACCESSORY,
    ['2 контроллера', Accessory(number=2, type='джойстик')],
    ['один мув', Accessory(number=1, type='move')]
)

## Storage

In [13]:
Storage = fact(
    'Storage',
    ['volume', 'unit']
)


INT = type('INT')

VOLUME = INT.interpretation(
    Storage.volume.custom(int)
)

UNITS = {
    'гб': 'GB',
    'gb': 'GB',
    'g': 'GB',

    'тб': 'TB',
    'tb': 'TB'
}

UNIT = in_caseless(UNITS).interpretation(
    Storage.unit.normalized().custom(UNITS.get) 
)

STORAGE = rule(
    VOLUME,
    UNIT
).interpretation(
    Storage
)


test(
    STORAGE,
    ['3гб', Storage(volume=3, unit='GB')],
    ['4G', Storage(volume=4, unit='GB')]
)

## Color

In [14]:
Color = fact(
    'Color',
    ['value']
)

MAPPING = {
    'black': 'чёрный',
    'white': 'белый',
    'green': 'зелёный',
}

EN_COLOR = in_caseless(MAPPING).interpretation(
    Color.value.normalized().custom(MAPPING.get)
)

COLOR = dictionary({
    'розовый',
    'белый',
    'чёрный',
    'синий',
    'фиолетовый',
    'красный',
    'оранжевый',
}).interpretation(
    Color.value.normalized()
)

COLOR = or_(
    EN_COLOR,
    COLOR
).interpretation(
    Color
)

test(
    COLOR,
    ['черным', Color(value='чёрный')],
    ['black', Color(value='чёрный')]
)

## Patch

In [15]:
Patched = fact(
    'Patched',
    [attribute('patched', False)]
)


PATCHED = normalized('прошить')

PATCHED = PATCHED.interpretation(
    Patched.patched.const(True)
).interpretation(
    Patched
)

## Console accesory

In [16]:
ConsoleAccessory = fact(
    'ConsoleAccessory',
    ['accessory', 'console']
)


FOR = in_caseless({
    'от',
    'для'
})

CONSOLE_ACCESSORY = rule(
    or_(
        ACCESSORY,
        GAMES
    ).interpretation(
        ConsoleAccessory.accessory
    ),
    rule(
        FOR.optional(),
        CONSOLE.interpretation(
            ConsoleAccessory.console
        )
    ).optional()
).interpretation(
    ConsoleAccessory
)


test(
    CONSOLE_ACCESSORY,
    [
        'Блок питание от Xbox360fat',
        ConsoleAccessory(
            accessory=Accessory(type='блок питания'),
            console=Console(name='Xbox', model='360', version='Fat')
        )
    ],
    [
        'Наушники для ps4',
        ConsoleAccessory(
            accessory=Accessory(type='наушники'),
            console=Console(name='PlayStation', model='4', version=None)
        )
    ],
    [
        '2 диска для PSP',
        ConsoleAccessory(
            accessory=Games(number=2), 
            console=Console(name='PSP')
        )
    ],
)

## Attributed console

In [17]:
AttributedConsole = fact(
    'AttributedConsole',
    ['console', attribute('attributes').repeatable()]
)

    
PLUS = in_caseless({
    '+',
    'плюс'
})

ATTRIBUTE = or_(
    GAMES,
    STORAGE,
    COLOR,
    PATCHED,
    CONSOLE_ACCESSORY,
).interpretation(
    AttributedConsole.attributes
)

ATTRIBUTE = rule(
    PLUS.optional(),
    ATTRIBUTE
)

ATTRIBUTED_CONSOLE = rule(
    CONSOLE.interpretation(
        AttributedConsole.console
    ),
    ATTRIBUTE.repeatable().optional()
).interpretation(
    AttributedConsole
)


test(
    ATTRIBUTED_CONSOLE,
    [
        'Sony PlayStation 3 Super Slim 500 GB + 5 игр прошитая',
        AttributedConsole(
            console=Console(vendor='Sony', name='PlayStation', model='3', version='SuperSlim'),
            attributes=[
                Storage(volume=500, unit='GB'),
                Games(number=5),
                Patched(patched=True)
            ]
        )
    ]
)

## Title

In [18]:
Title = fact(
    'Title',
    ['item']
)


TITLE = or_(
    ATTRIBUTED_CONSOLE,
    CONSOLE_ACCESSORY
).interpretation(
    Title.item
).interpretation(
    Title
)

test(
    TITLE,
    [
        'PS 4 white 500g',
        Title(item=AttributedConsole(
            console=Console(name='PlayStation', model='4'),
            attributes=[
                Color(value='белый'),
                Storage(volume=500, unit='GB')
            ]
        ))
    ],
    [
        'Геймпад джойстик для ps 3',
        Title(item=ConsoleAccessory(
            accessory=Accessory(type='джойстик'),
            console=Console(name='PlayStation', model='3')
        ))
    ]
)

# Extractor

In [19]:
import json

from ipymarkup import show_span_ascii_markup as show_markup


def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )


class Match(object):
    def __init__(self, fact, spans):
        self.fact = fact
        self.spans = spans


    
DEBUG = or_(
    CONSOLE,
    ACCESSORY,
    rule(FOR),
    rule(PLUS),
    GAMES,
    STORAGE,
    COLOR,
    PATCHED,
)


class Extractor(object):
    def __init__(self):
        self.debug = Parser(DEBUG)
        self.parser = Parser(TITLE)

    def __call__(self, line):
        matches = self.debug.findall(line)
        spans = [_.span for _ in matches]
        line = join_spans(line, spans)
        matches = self.parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        fact = None
        if matches:
            fact = matches[0].fact.item
        return Match(fact, spans)
    
    
# 6 ошибок, 8 неполных из 100
extractor = Extractor()
seed(10)
for line in sample(lines, 100):
    match = extractor(line)
    show_markup(line, match.spans)
    if match.fact:
        show_json(match.fact.as_json)

Пульт PlayStation3
───── ────────────
{
  "accessory": {
    "type": "пульт"
  },
  "console": {
    "name": "PlayStation",
    "model": "3"
  }
}
Сонька 3
────────
{
  "console": {
    "name": "PlayStation",
    "model": "3"
  },
  "attributes": []
}
Xbox one
────────
{
  "console": {
    "name": "Xbox",
    "model": "One"
  },
  "attributes": []
}
Sony PlayStation 4 500Gb Slim
────────────────── ─────     
{
  "console": {
    "vendor": "Sony",
    "name": "PlayStation",
    "model": "4"
  },
  "attributes": [
    {
      "volume": 500,
      "unit": "GB"
    }
  ]
}
Новый жесткий кейс psp
              ──── ───
{
  "accessory": {
    "type": "чехол"
  },
  "console": {
    "name": "PSP"
  }
}
Ps3 slim прошитая rebug 4.81 + fifa 18
──────── ────────            ─        
{
  "console": {
    "name": "PlayStation",
    "model": "3",
    "version": "Slim"
  },
  "attributes": [
    {
      "patched": true
    }
  ]
}
Беспроводной геймпад для Xbox One
──────────────────── ─── ────────
{


──────── ─ ──────────   ─────
{
  "console": {
    "name": "Xbox",
    "model": "360"
  },
  "attributes": [
    {
      "accessory": {
        "number": 2,
        "type": "джойстик"
      }
    },
    {
      "number": 5
    }
  ]
}
Xbox 360, 250 гб
────────  ──────
{
  "console": {
    "name": "Xbox",
    "model": "360"
  },
  "attributes": [
    {
      "volume": 250,
      "unit": "GB"
    }
  ]
}
Sony PlayStation 4 Pro 1tb
────────────────────── ───
{
  "console": {
    "vendor": "Sony",
    "name": "PlayStation",
    "model": "4",
    "version": "PRO"
  },
  "attributes": [
    {
      "volume": 1,
      "unit": "TB"
    }
  ]
}
PSP Slim 3008 Прошитая и 3004
────────      ────────       
{
  "console": {
    "name": "PSP",
    "model": "Slim"
  },
  "attributes": [
    {
      "patched": true
    }
  ]
}
Xbox 360 E 500Gb на гарантии
────────── ─────            
{
  "console": {
    "name": "Xbox",
    "model": "360",
    "version": "E"
  },
  "attributes": [
    {
      "volume"